#### Auto reload imports to ease work with python modules

In [1]:
%load_ext autoreload
%autoreload 2

#### Import Python packages 

In [2]:
import pandas as pd

from data_processing import (generate_base_csv, get_file_path_list, get_values_1, get_values_2,
                             get_values_3, read_csv_and_insert_in_table)
from queries import (CREATE_TABLE_1, CREATE_TABLE_2, CREATE_TABLE_3, INSERT_1, INSERT_2, INSERT_3,
                     QUERY_1, QUERY_2, QUERY_3)
from tools import CassandraHandler


## Part I. ETL Pipeline for Pre-Processing the Files

In [3]:
FILE = 'event_datafile_new.csv'


Create list of filepaths to process original event csv data files

In [4]:

file_path_list = get_file_path_list()
file_path_list[0]

/home/gui/Projects/de-02-data-modeling-with-cassandra
Number of file paths: 10


'/home/gui/Projects/de-02-data-modeling-with-cassandra/event_data/2018-11-03-events.csv'

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
generate_base_csv(file_path_list, FILE)


len(full_data_rows_list): 1882

full_data_rows_list[0]: ['', 'Logged Out', '', '', '0', '', '', 'free', '', 'PUT', 'Login', '', '52', '', '307', '1.54121E+12', '']


View dataset

In [6]:
df = pd.read_csv(FILE)
df.head()

,artist,first_name,gender,item_in_session,last_name,length,level,location,session_id,song,user_id
0,Mynt,Celeste,F,2,Williams,166.94812,free,"Klamath Falls, OR",52,Playa Haters,53
1,Taylor Swift,Celeste,F,3,Williams,230.47791,free,"Klamath Falls, OR",52,You Belong With Me,53
2,Amy Winehouse,Celeste,F,4,Williams,229.85098,free,"Klamath Falls, OR",52,Valerie,53
3,Jimmy Eat World,Celeste,F,5,Williams,285.83138,free,"Klamath Falls, OR",52,Dizzy,53
4,Maldita Nerea,Anabelle,F,0,Simpson,241.16200,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",158,Supelicula,69


#### Cassandra Operations

Create Cluster and Session

In [7]:

cluster, session = CassandraHandler.create_cluster_and_session()


Create Keyspace

In [8]:
CassandraHandler.create_keyspace(session)


Set Keyspace

In [9]:
CassandraHandler.set_keyspace(session, 'udacity')


**Query 1** - Create table, insert into it, and check query
*Give me the artist, song title and song's length in the music app history
that was heard during sessionId = 338, and itemInSession = 4*

In [10]:
CassandraHandler.execute(session, CREATE_TABLE_1)
read_csv_and_insert_in_table(FILE, session, INSERT_1, get_values_1)
CassandraHandler.get_query_results(session, QUERY_1)

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


**Query 2** - Create table, insert into it, and check query


*Give me only the following: name of artist, song (sorted by itemInSession), 
user (first and last name) for userid = 10, sessionid = 182*

In [11]:
CassandraHandler.execute(session, CREATE_TABLE_2)
read_csv_and_insert_in_table(FILE, session, INSERT_2, get_values_2)
CassandraHandler.get_query_results(session, QUERY_2)


,artist,song_title,first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


**Query 3** - Create table, insert into it, and check query

*Give me every user name (first and last) in my music app history who listened
to the song 'All Hands Against His Own'*

In [12]:
CassandraHandler.execute(session, CREATE_TABLE_3)
read_csv_and_insert_in_table(FILE, session, INSERT_3, get_values_3)
CassandraHandler.get_query_results(session, QUERY_3)


,first_name,user_last_name
0,Sara,Johnson


Drop the tables before closing out the sessions

In [13]:
CassandraHandler.drop_table(session, "event_1")
CassandraHandler.drop_table(session, "event_2")
CassandraHandler.drop_table(session, "event_3")


Close the session and cluster connection

In [14]:
CassandraHandler.close_cluster_and_session(cluster, session)